In [1]:
import talib


In [2]:
import numpy as np
import pandas as pd

In [3]:
teste = np.random.random(40)

In [4]:
teste

array([0.10677447, 0.09491487, 0.79595187, 0.58486099, 0.3369761 ,
       0.58064954, 0.49565002, 0.63300167, 0.91552666, 0.24612084,
       0.44704609, 0.3646556 , 0.26218218, 0.42483388, 0.34422951,
       0.96122447, 0.51043235, 0.67337378, 0.42269872, 0.1276166 ,
       0.56662624, 0.44789215, 0.71511154, 0.42146539, 0.45012137,
       0.33275315, 0.04440951, 0.26016603, 0.6422415 , 0.1608384 ,
       0.32503253, 0.54330493, 0.52151253, 0.11427884, 0.81487772,
       0.96606785, 0.28215813, 0.25141266, 0.10745555, 0.08928521])

In [5]:
sma = talib.SMA(teste)
print(sma)

[       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan 0.44567818
 0.45295345 0.46789979 0.45875181 0.44306574 0.45899579 0.47184307
 0.46472667 0.45200704 0.42507133 0.41984348]
